<a href="https://colab.research.google.com/github/XaxUser/Projects/blob/main/smolagents_doc/en/agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
! pip install torch streamlit pdfplumber python-dotenv langgraph langchain chromadb langchain_chroma langchain_community langchain_experimental chunking langchain_docling google-generativeai langchain_google_genai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 1.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is looking at multiple versions of langchain-google-genai to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 5.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 61.8 M

In [3]:
! pip install streamlit

In [5]:
! pip install fastembed

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.0/99.0 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.8/324.8 kB 11.3 MB/s eta 0:00:00


In [6]:
import sys
import types

# Patch pour éviter que streamlit ne tente de parcourir torch.classes
sys.modules['torch.classes'] = types.ModuleType('torch.classes')
import os
import time
import ssl
import pickle
import hashlib
import warnings
import json
import logging
import concurrent.futures
from io import StringIO
from typing import Any, Dict, List, Tuple, TypedDict, Callable
from functools import lru_cache
from google.colab import userdata
import streamlit as st
import pdfplumber
from dotenv import load_dotenv

# LangGraph & LangChain imports
from langgraph.graph import StateGraph, END
from langgraph.prebuilt import ToolNode
from langgraph.checkpoint.memory import MemorySaver
from langchain.tools import Tool
from chromadb.config import Settings
from langchain_chroma import Chroma
from langchain_community.embeddings import FastEmbedEmbeddings
from langchain_community.chat_models import ChatOllama
from langchain.schema import Document
from langchain_experimental.text_splitter import SemanticChunker
from langchain_core.messages import SystemMessage, HumanMessage
from langchain.prompts import ChatPromptTemplate
#from chunking import ClusterSemanticChunker
from chromadb.utils import embedding_functions
#from langchain_docling import DoclingLoader
from langchain_docling.loader import DocumentConverter
import google.generativeai as genai
from langchain_google_genai import ChatGoogleGenerativeAI

api_key = userdata.get('GOOGLE_API_KEY')
os.environ["GOOGLE_API_KEY"] = api_key
genai.configure(api_key=api_key)

# --- Load environment variables ---
load_dotenv()

# --- Silence deprecation warnings ---
warnings.filterwarnings("ignore", category=DeprecationWarning)

# --- SSL certificate workaround ---
try:
    ssl._create_default_https_context = ssl._create_unverified_context
except AttributeError:
    pass

# --- Logging configuration ---
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[logging.StreamHandler(), logging.FileHandler("matching_app.log")]
)
logger = logging.getLogger(__name__)

## --- Directories ---
AGENT_STORAGE_DIR = "storage"
#KB_DIR = os.path.join(AGENT_STORAGE_DIR, "pdf_knowledge_base")
#CACHE_DIR = os.path.join(AGENT_STORAGE_DIR, "cache")
#EMBED_CACHE = os.path.join(CACHE_DIR, "embeddings")
#CHUNKS_CACHE = os.path.join(CACHE_DIR, "chunks")
#for d in [AGENT_STORAGE_DIR, KB_DIR, CACHE_DIR, EMBED_CACHE, CHUNKS_CACHE]:
#    os.makedirs(d, exist_ok=True)

# --- Typed state dict ---
class AgentState(TypedDict, total=False):
    messages: List[Dict[str, Any]]
    context: str
    ao_skills: str
    cv_skills: str

# --- Caching utils ---
#def get_file_hash(path: str) -> str:
#    """
#    Calcule le hash SHA-256 d'un fichier pour le caching.
#    Optimisé pour les gros fichiers avec lecture par chunks.
#    """
#    hasher = hashlib.sha256()
#    try:
#        with open(path, 'rb') as f:
#            while chunk := f.read(65536):
#                hasher.update(chunk)
#        return hasher.hexdigest()
#    except Exception as e:
#        logger.error(f"Erreur lors du calcul du hash pour {path}: {e}")
#        return hashlib.sha256(path.encode()).hexdigest()  # Fallback
#
#def cache_path(path: str, kind: str) -> str:
#    """
#    Génère un chemin de cache basé sur le hash du fichier.
#    """
#    h = get_file_hash(path)[:8]
#    name = os.path.splitext(os.path.basename(path))[0]
#    folder = CHUNKS_CACHE if kind == 'chunks' else EMBED_CACHE
#    return os.path.join(folder, f"{name}_{h}_{kind}.pkl")
#
#def save_cache(data: Any, path: str) -> None:
#    """
#    Sauvegarde des données dans le cache avec gestion d'erreurs.
#    """
#    try:
#        os.makedirs(os.path.dirname(path), exist_ok=True)
#        with open(path, 'wb') as f:
#            pickle.dump(data, f)
#        logger.debug(f"Cache sauvegardé: {path}")
#    except Exception as e:
#        logger.warning(f"Erreur lors de la sauvegarde du cache {path}: {e}")
#
#def load_cache(path: str) -> Any:
#    """
#    Charge des données depuis le cache avec gestion d'erreurs.
#    """
#    try:
#        if os.path.exists(path):
#            with open(path, 'rb') as f:
#                data = pickle.load(f)
#            logger.debug(f"Cache chargé: {path}")
#            return data
#    except Exception as e:
#        logger.warning(f"Erreur lors du chargement du cache {path}: {e}")
#    return None

def lire_mkd(chemin):
    """
    Lit un fichier .mkd et retourne son contenu sous forme de chaîne de caractères.

    Paramètre
    ---------
    chemin : str
        Chemin vers le fichier .mkd à lire.

    Retour
    ------
    str
        Contenu du fichier Markdown.

    Exemple
    -------
    >>> contenu = lire_mkd("mon_document.mkd")
    >>> print(contenu)
    "# Titre\n\nVoici le contenu Markdown..."
    """
    # Vérifier que le fichier existe
    if not os.path.isfile(chemin):
        raise FileNotFoundError(f"Le fichier '{chemin}' n'existe pas.")
    # Vérifier que l'extension est bien .mkd
    if not chemin.lower().endswith(".mkd"):
        raise ValueError(f"Le fichier '{chemin}' n'a pas l'extension .mkd.")

    # Lecture du fichier en UTF-8 (adapter l'encodage si besoin)
    with open(chemin, "r", encoding="utf-8") as f:
        return f.read()

# --- PDF extraction ---
def extract_text_and_tables(pdf_path: str) -> List[Tuple[str, str]]:
    """
    Extrait le texte et les tableaux d'un PDF avec multithreading.
    Optimisé pour la performance et la robustesse.
    """
    all_contents = []

    with pdfplumber.open(pdf_path) as pdf:
        for i, page in enumerate(pdf.pages):
            if i == 1:
                continue
            text = page.extract_text()
            if "6. CONDITIONS" in text:
                L = text.split("6. CONDITIONS")
                if len(L)>1:
                    all_contents.append(("texte", L[0].strip()))
                break
            text = text.replace("© Copyright Orange 2025","")
            text = text.replace("Document confidentiel","")
            text = text.replace("La communication de ce document est soumise à autorisation d'Orange","")
            if text:
                all_contents.append(("texte", text.strip()))

            tables = page.extract_tables()
            for table in tables:
                # Nettoyer le tableau et le transformer en texte structuré
                #table_text = "\n".join([" | ".join(cell.strip() for cell in row if cell) for row in table])
                table_text = ""
                headers = table[0]
                for row in table[1:]:
                    for header, cell in zip(headers, row):
                        if header and cell:
                            table_text += f"{header.strip()} : {cell.strip()}\n"
                    table_text += "\n"
                if table_text:
                    all_contents.append(("tableau", table_text.strip()))

    return all_contents

# --- Embeddings & vector stores ---
# Configuration des embeddings avec cache
embeddings = FastEmbedEmbeddings(
    #model_name="sentence-transformers/all-MiniLM-L6-v2",
    #cache_folder=EMBED_CACHE,
    max_seq_length=512,
    #doc_embed_type="passage"
)

# Configuration de ChromaDB
settings = Settings(
    anonymized_telemetry=False,
    allow_reset=True,  # Permettre la réinitialisation des collections
    is_persistent=False
)

# --- Semantic chunking ---
def create_semantic_chunks(texts: List[str], file_path: str) -> List[Document]:
    """
    Crée des chunks sémantiques à partir des textes extraits.
    Utilise le cache pour éviter de retraiter les mêmes documents.
    Optimisé pour différentes tailles de documents.
    """
    # Vérifier le cache
    #cpath = cache_path(file_path, 'chunks')
    #cached = load_cache(cpath)
    #if cached:
    #    logger.info(f"Utilisation du cache pour les chunks: {file_path}")
    #    return cached

    # Adapter les paramètres de chunking en fonction de la taille du document
    total_len = sum(len(t) for t in texts)
    logger.info(f"Taille totale du texte: {total_len} caractères")

    if total_len < 10000:
        max_size, min_size = 300, 100
    elif total_len < 50000:
        max_size, min_size = 500, 150
    else:
        max_size, min_size = 12000, 200

    try:
        # Créer les chunks avec gestion d'erreurs
        splitter = SemanticChunker(
            embeddings=embeddings,
            min_chunk_size=min_size,
            #add_start_index=True,  # Ajouter l'index de début pour faciliter le débogage
            #breakpoint_threshold_type='percentile',  # Méthode robuste pour découpage
            #breakpoint_threshold_amount=5,
            #sentence_split_regex= r"(?<=[.?!])\s+#"
        )

        #splitter = ClusterSemanticChunker(
        #    embedding_function=embedding_functions.DefaultEmbeddingFunction(),
        #    max_chunk_size=1200,
        #    min_chunk_size=400
        #) #800-400 70

        # Filtrer les textes vides ou trop courts
        #print(texts)
        if len(texts[0]) >10:
            valid_texts = texts[0]
        else:
            valid_texts = texts # and len(t.strip()) > 20]
        #print(valid_texts)
        if not valid_texts:
            logger.warning(f"Aucun texte valide à chunker pour {file_path}")
            return []

        # Créer les documents
        #docs_raw = splitter.create_documents(valid_texts)
        docs_raw = splitter.split_text(valid_texts)
        #docs_raw = "\n\n".join(valid_texts)
        #docs_raw = valid_texts.split("##")
        valid_texts = [text for text in valid_texts if len(text.strip()) > 20]
        #print(docs_raw)
        logger.info(f"Chunks créés: {len(docs_raw)} chunks")
        docs = [Document(page_content=chunk, metadata={"source": os.path.basename(file_path)}) for chunk in docs_raw]

        # Vérifier que les chunks ont été créés correctement
        if not docs:
            logger.warning(f"Aucun chunk créé pour {file_path}")
            return []

        # Sauvegarder dans le cache
        #save_cache(docs, cpath)
        logger.info(f"Chunks créés et mis en cache: {len(docs)} chunks")

        return docs

    except Exception as e:
        logger.error(f"Erreur lors du chunking pour {file_path}: {e}", exc_info=True)
        return []

# --- Gestion améliorée des collections ChromaDB ---
def create_or_get_collection(name: str) -> Chroma:
    """
    Crée une nouvelle collection ChromaDB ou récupère une existante.
    Gère les erreurs de manière robuste avec retry et backoff exponentiel.
    """
    collection_id = hashlib.md5(name.encode()).hexdigest()

    logger.info(f"Tentative de création/récupération de la collection '{name}' (ID: {collection_id})")

    try:
        # Tentative de création d'une nouvelle collection
        store = Chroma.from_documents(
            documents=[],  # Collection vide
            embedding=embeddings,
            collection_name=collection_id,  # Utiliser un ID stable basé sur le hash du nom
            client_settings=settings
        )

        # Vérifier que la collection est accessible
        try:
            count = len(store.get())
            logger.info(f"Collection '{name}' (ID: {collection_id}) créée/récupérée avec succès: {count} documents")
            return store
        except Exception as inner_e:
            logger.warning(f"Collection '{name}' créée mais non accessible: {inner_e}")
            # Continuer pour réessayer

    except Exception as e:
        logger.warning(f"Erreur avec collection '{name}': {e}")

    # Si toutes les tentatives ont échoué, créer une collection minimale en dernier recours
    logger.warning(f"Création d'une collection minimale '{name}' après échecs multiples")
    return Chroma(
        collection_name=collection_id,
        embedding_function=embeddings,
        client_settings=settings
    )


# Initialisation des stores avec la nouvelle fonction robuste
global vector_store_AO
vector_store_AO = create_or_get_collection(f"AO_documents")
global vector_store_CV
vector_store_CV = create_or_get_collection(f"CV_documents")

# --- PDF ingestion ---
def ingest_pdf(path: str, dtype: str) -> str:
    """
    Fonction d'ingestion PDF améliorée avec gestion d'erreurs robuste.
    Optimisée pour la performance et la fiabilité.
    """

    start_time = time.time()
    logger.info(f"Début d'ingestion du PDF {path} (type: {dtype})")

    # Validation des entrées
    if not os.path.exists(path):
        return f"Erreur: {path} introuvable"
    #if not path.lower().endswith('.mkd'):
    #    return "Erreur: fournir un PDF"

    # Sélection du store approprié
    store_name = "AO_documents" if dtype=='AO' else "CV_documents"
    collection_id = hashlib.md5(store_name.encode()).hexdigest()

    try:
        # Récréer la collection
        if dtype == 'AO':
            global vector_store_AO
            vector_store_AO = reset_vector_store("AO_documents")
            store = vector_store_AO
        else:
            global vector_store_CV
            vector_store_CV = reset_vector_store("CV_documents")
            store = vector_store_CV

        # Extraction du contenu avec mesure de performance
        extract_start = time.time()
        logger.info(f"Extraction du contenu du PDF {path}...")
        #content = extract_text_and_tables(path)
        #content = DocumentConverter().convert(path)  #.document.export_to_markdown()
        #content = [content.document.export_to_markdown()]
        if path.lower().endswith('.pdf'):
            content = DocumentConverter().convert(path)  #.document.export_to_markdown()
            content = [content.document.export_to_markdown()]
        else:
            content = lire_mkd(path)
        extract_time = time.time() - extract_start
        logger.info(f"Extraction terminée en {extract_time:.2f}s: {len(content)} éléments extraits")

        if not content:
            return f"Aucun contenu extrait de {os.path.basename(path)}"

        # Création des chunks avec mesure de performance
        chunk_start = time.time()
        logger.info(f"Création des chunks sémantiques...")
        texts = content # [c for c in content]
        docs = create_semantic_chunks(texts, path)
        chunk_time = time.time() - chunk_start
        logger.info(f"Chunking terminé en {chunk_time:.2f}s: {len(docs)} chunks créés")

        if not docs:
            return f"Chunking échoué pour {os.path.basename(path)}"

        # Ajout des documents avec mesure de performance
        embed_start = time.time()
        logger.info(f"Ajout des documents au vector store...")

        # Ajout par lots pour éviter les timeouts
        batch_size = 50
        for i in range(0, len(docs), batch_size):
            batch = docs[i:i+batch_size]
            try:
                store.add_documents(batch)
                logger.info(f"Lot {i//batch_size + 1}/{(len(docs)-1)//batch_size + 1} ajouté ({len(batch)} docs)")
            except Exception as e:
                logger.error(f"Erreur lors de l'ajout du lot {i//batch_size + 1}: {e}")
                # Continuer avec le lot suivant

        embed_time = time.time() - embed_start
        logger.info(f"Embedding et indexation terminés en {embed_time:.2f}s")

        # Vérification finale
        try:
            count = len(store.get())
            logger.info(f"Documents dans {store_name} (ID: {collection_id}): {count}")

            if count == 0:
                return f"Avertissement: Aucun document n'a été ajouté à {store_name}"

        except Exception as e:
            logger.warning(f"Impossible de vérifier le nombre de documents: {e}")

        total_time = time.time() - start_time
        return f"Ingestion réussie de {os.path.basename(path)} en {total_time:.2f}s"

    except Exception as e:
        logger.error(f"Ingestion failed for {path}", exc_info=True)
        return f"Erreur: {str(e)}"

def ingest_pdf_document_AO(fp: str) -> str:
    return ingest_pdf(fp, 'AO')

def ingest_pdf_document_CV(fp: str) -> str:
    return ingest_pdf(fp, 'CV')

# --- LLM setup ---
@lru_cache(maxsize=1)
def get_llm(temperature=0.7, top_p=0.9, top_k=40):
    """
    Obtient une instance du LLM avec cache pour éviter les réinitialisations inutiles.
    Paramètres optimisés pour différentes tâches.
    """
    try:
        return ChatGoogleGenerativeAI(
            model="gemini-1.5-flash",  # Ou "gemini-2.5-pro" si disponible via l'API
            temperature=temperature,
            top_p=top_p,
            top_k=top_k,
            convert_system_message_to_human=True  # selon les besoins
        )
    except Exception as e:
        logger.error(f"Erreur lors de l'initialisation du LLM: {e}")
        # Fallback avec paramètres minimaux
        return ChatOllama(model="gemma3:1b")

# --- Prompts améliorés avec Chain-of-Thought et exemples few-shot ---

#AO_SYSTEM_PROMPT = """
#Vous êtes un expert en analyse de documents d'appels d'offres (AO), spécialisé dans l'extraction et la catégorisation des compétences, technologies et exigences.
#
#**Votre tâche:**
#1. **Extraire** toutes les compétences techniques, soft skills, technologies, frameworks, certifications et exigences d'expérience.
#2. **Catégoriser** chaque élément en:
#   - **Compétences techniques** (ex: Python, Docker)
#   - **Soft skills** (ex: Gestion de projet, Leadership d'équipe)
#   - **Certifications** (ex: PMP, AWS Certified)
#   - **Exigences d'expérience** (ex: "5+ ans en architecture cloud")
#3. **Évaluer** l'importance de chaque élément:
#   - **Critique** (obligatoire, mentionné comme essentiel)
#   - **Important** (fortement souhaité mais pas obligatoire)
#   - **Souhaitable** (mentionné comme un plus)
#
#**Processus d'analyse:**
#1. D'abord, lisez attentivement tout le contexte fourni
#2. Identifiez toutes les compétences et exigences mentionnées
#3. Pour chaque compétence, déterminez sa catégorie
#4. Évaluez l'importance de chaque compétence en fonction du contexte
#5. Organisez les résultats par catégorie et importance
#
#**Format de sortie:**
#```
### Compétences techniques
#- [Nom de la compétence] (Critique/Important/Souhaitable): [Description ou exigence spécifique]
#
### Soft skills
#- [Nom de la compétence] (Critique/Important/Souhaitable): [Description ou exigence spécifique]
#
### Certifications
#- [Nom de la certification] (Critique/Important/Souhaitable): [Description ou exigence spécifique]
#
### Exigences d'expérience
#- [Description de l'expérience] (Critique/Important/Souhaitable): [Détails supplémentaires]
#```
#
#**Exemple:**
#```
### Compétences techniques
#- Python (Critique): Maîtrise de Python 3.8+ pour le développement backend
#- Docker (Important): Expérience avec la conteneurisation et Docker Compose
#- React (Souhaitable): Connaissance de base pour collaborer avec l'équipe frontend
#
### Soft skills
#- Gestion de projet (Critique): Capacité à gérer des projets Agile de bout en bout
#- Communication (Important): Excellentes compétences en communication écrite et orale
#
### Certifications
#- AWS Solutions Architect (Souhaitable): Certification professionnelle préférée
#
### Exigences d'expérience
#- Développement cloud (Critique): Minimum 3 ans d'expérience en développement d'applications cloud
#```
#
#Basez votre analyse UNIQUEMENT sur le contenu du document fourni. N'inventez pas d'informations qui ne sont pas explicitement mentionnées.
#"""
#
#CV_SYSTEM_PROMPT = """
#Vous êtes un expert en analyse de CV (Curriculum Vitae), spécialisé dans l'extraction et l'évaluation des compétences et expériences professionnelles.
#
#**Votre tâche:**
#1. **Extraire** toutes les compétences techniques, soft skills, certifications et expériences pertinentes du CV.
#2. **Évaluer** le niveau de maîtrise pour chaque compétence sur une échelle de 1 à 4:
#   - **1 = Basique**: Connaissance fondamentale, peu d'expérience pratique
#   - **2 = Intermédiaire**: Bonne connaissance, quelques projets/expériences
#   - **3 = Avancé**: Maîtrise solide, expérience significative
#   - **4 = Expert**: Expertise approfondie, longue expérience, contributions notables
#3. **Estimer** un pourcentage de confiance pour chaque évaluation.
#
#**Processus d'analyse:**
#1. D'abord, lisez attentivement tout le CV
#2. Identifiez toutes les compétences mentionnées explicitement
#3. Déduisez les compétences implicites à partir des expériences décrites
#4. Pour chaque compétence:
#   - Évaluez le niveau en fonction de la durée d'expérience, des projets réalisés et du contexte
#   - Estimez un pourcentage de confiance basé sur la clarté des informations
#5. Organisez les résultats par catégorie
#
#**Format de sortie:**
#```
### Compétences techniques
#- [Nom de la compétence] - [Niveau]/4: [Justification basée sur le CV]
#
### Soft skills
#- [Nom de la compétence] - [Niveau]/4: [Justification basée sur le CV]
#
### Certifications
#- [Nom de la certification] - [Année obtenue si mentionnée]
#
### Expériences clés
#- [Description concise de l'expérience] - [Durée]: [Compétences démontrées]
#```
#
#**Exemple:**
#```
### Compétences techniques
#- Python - 3/4: 5 ans d'expérience, développement de plusieurs applications backend
#- Docker - 2/4: Mentionné dans 2 projets récents, utilisation basique
#- AWS - 4/4 : Certification AWS, 7 ans d'expérience, architecture de solutions complexes
#
### Soft skills
#- Gestion de projet - 3/4: A dirigé 3 équipes de développement, méthodologie Agile
#- Communication - 2/4: Présentations clients mentionnées, pas de détails spécifiques
#
### Certifications
#- AWS Solutions Architect Professional - 2022
#- Scrum Master Certified - 2020
#
### Expériences clés
#- Architecte Cloud chez TechCorp - 3 ans: Conception et implémentation d'infrastructures AWS
#- Lead Developer chez StartupXYZ - 2 ans: Développement Python, gestion d'équipe
#```
#
#Basez votre analyse UNIQUEMENT sur le contenu du CV fourni. N'inventez pas d'informations qui ne sont pas explicitement ou implicitement présentes dans le document.
#"""
#
#MATCHER_SYSTEM_PROMPT = """
#Vous êtes un expert en matching de compétences, spécialisé dans la comparaison des exigences d'un Appel d'Offres (AO) avec les compétences présentes dans un CV.
#
#**Votre tâche:**
#1. **Analyser** les compétences requises dans l'AO et celles présentes dans le CV
#2. **Identifier** les correspondances, les lacunes et les atouts supplémentaires
#3. **Évaluer** la qualité de chaque correspondance
#4. **Fournir** une analyse détaillée et des recommandations
#
#**Processus d'analyse:**
#1. D'abord, examinez attentivement les deux listes de compétences
#2. Pour chaque compétence requise dans l'AO:
#   - Recherchez la compétence correspondante dans le CV
#   - Évaluez si le niveau de compétence du CV répond à l'exigence de l'AO
#   - Tenez compte de l'importance de la compétence dans l'AO
#3. Identifiez les compétences critiques manquantes
#4. Repérez les compétences supplémentaires du CV qui pourraient être valorisées
#5. Calculez un score global d'adéquation
#
#**Format de sortie:**
#```
## Analyse de correspondance AO-CV
#
### Score global d'adéquation: [X]%
#
### Correspondances fortes
#- [Compétence]: AO requiert [niveau d'importance], CV montre [niveau de maîtrise] (Excellente correspondance)
#  > Justification: [Explication détaillée]
#
### Correspondances acceptables
#- [Compétence]: AO requiert [niveau d'importance], CV montre [niveau de maîtrise] (Correspondance suffisante)
#  > Justification: [Explication détaillée]
#
### Correspondances faibles
#- [Compétence]: AO requiert [niveau d'importance], CV montre [niveau de maîtrise] (Amélioration nécessaire)
#  > Justification: [Explication détaillée]
#  > Recommandation: [Suggestion pour combler l'écart]
#
### Compétences critiques manquantes
#- [Compétence]: AO requiert [niveau d'importance] (Absente du CV)
#  > Impact: [Évaluation de l'impact sur la candidature]
#  > Recommandation: [Suggestion pour combler cette lacune]
#
### Atouts supplémentaires
#- [Compétence]: CV montre [niveau de maîtrise] (Non requise mais valorisable)
#  > Valeur ajoutée: [Explication de la valeur potentielle]
#
### Recommandation finale
#[Analyse synthétique de l'adéquation globale et recommandations stratégiques]
#```
#
#**Exemple de raisonnement:**
#1. L'AO demande "Python (Critique)" et le CV montre "Python - 3/4"
#   → C'est une correspondance forte car le niveau est avancé pour une exigence critique
#2. L'AO demande "AWS (Important)" mais le CV ne mentionne pas AWS
#   → C'est une compétence importante manquante qui pourrait affecter la candidature
#3. Le CV mentionne "Docker - 4/4" qui n'est pas explicitement requis dans l'AO
#   → C'est un atout supplémentaire qui pourrait être valorisé
#
#Basez votre analyse UNIQUEMENT sur les informations fournies dans les listes de compétences. Soyez précis, objectif et constructif dans vos évaluations.
#"""

#AO_SYSTEM_PROMPT = """
#Vous êtes un expert en AO. À partir d’un document, extrayez et classez :
#
#1. Compétences techniques (p. ex. Python, Docker)
#2. Soft skills (p. ex. gestion de projet, leadership)
#3. Certifications (p. ex. PMP, AWS Certified)
#4. Exigences d’expérience (p. ex. « 5+ ans cloud »)
#
#Pour chaque élément, indiquez :
#- Catégorie
#- Niveau d’importance : Critique (obligatoire), Important (fortement souhaité), Souhaitable (atout)
#- Détail ou citation du texte source
#
#Répondez au format :
#
### Compétences techniques
#- [Nom] (Critique/Important/Souhaitable) : [Extrait de l’AO]
#
### Soft skills
#- …
#
### Certifications
#- …
#
### Exigences d’expérience
#- …
#Basez votre analyse UNIQUEMENT sur les informations fournies dans les listes de compétences. Soyez précis, objectif et constructif dans vos évaluations.
#"""
#
#CV_SYSTEM_PROMPT = """
#Vous êtes un expert en CV. À partir d’un CV, extrayez et évaluez :
#
#1. Compétences techniques
#2. Soft skills
#3. Certifications
#4. Expériences clés
#
#Pour chaque compétence, indiquez :
#- Niveau (1–4) : 1=basique, 2=intermédiaire, 3=avancé, 4=expert
#- Confiance (en %)
#- Justification tirée du CV
#
#Répondez au format :
#
### Compétences techniques
#- [Nom] – [Niveau]/4 (Confiance %) : [Justification]
#
### Soft skills
#- …
#
### Certifications
#- [Nom] – [Année]
#
### Expériences clés
#- [Poste, entreprise, durée] : [Compétences démontrées]
#Basez votre analyse UNIQUEMENT sur les informations fournies dans les listes de compétences. Soyez précis, objectif et constructif dans vos évaluations.
#"""
#
#MATCHER_SYSTEM_PROMPT = """
#Vous êtes un expert en matching AO–CV. À partir de deux listes (AO & CV), réalisez :
#
#1. Score global (%) d’adéquation
#2. Correspondances fortes
#3. Correspondances acceptables
#4. Correspondances faibles (avec recommandations)
#5. Compétences critiques manquantes (impact & recommandations)
#6. Atouts supplémentaires (valeur ajoutée)
#
#Pour chaque point, mentionnez :
#- Compétence
#- Importance AO vs. niveau CV
#- Justification
#
#Format de réponse :
#
## Matching AO–CV
#
### Score global : X %
#
#### Correspondances fortes
#- …
#
#### Correspondances acceptables
#- …
#
#### Correspondances faibles
#- …
#
#### Critiques manquantes
#- …
#
#### Atouts supplémentaires
#- …
#
#### Synthèse et recommandations finales
#Basez votre analyse UNIQUEMENT sur les informations fournies dans les listes de compétences. Soyez précis, objectif et constructif dans vos évaluations.
#"""

#AO_SYSTEM_PROMPT = """
#You are an expert document analysis agent specialized in extracting skills, technologies, frameworks, and other relevant information from documents, focusing on Call for Tenders (Appel d'Offres).
#
#Your goal is to provide accurate, reliable analysis based *only* on the provided document.
#
#1. Understand the user's query clearly.
#2. Strategically search relevant sections of the document using your knowledge base.
#3. Synthesize information from multiple parts if needed.
#4. Provide well-structured, concise answers.
#
#Focus your search on relevant sections mentioned (like 'section 1.2.3' or 'section 4.2' or 'header page') if specified in the query.
#
#Your responses should:
#- Be formatted as bullet points.
#- Include confidence percentages for each point (e.g., 'Python - 3/4').
#- Be concise but informative.
#- Only include information actually found in the document.
#- Never invent or assume information not explicitly stated.
#
#Always maintain a neutral, analytical tone.
#If the knowledge base is empty or the document hasn't been ingested, inform the user they need to ingest a PDF first using the 'ingest_pdf_document' tool with the file path.
#"""
#
#CV_SYSTEM_PROMPT = """
#You are an analytic system specialized in analyzing CV (Curriculum Vitae) documents.
#
#Your task is to analyse the CV and answer the question or task given to you.
#
#You have to avoid inventing new skills and only mention those in the file.
#You have to list the various skills mentioned in the file and evaluate the level of knowledge for such skills.
#Convert experiences into skills and evaluate the level of knowledge on a scale of 1 to 4 (1=Basic, 2=Working, 3=Advanced, 4=Expert).
#
#To answer, first analyse the query, then use the knowledge base to gather relevant data, create a temporary answer, review it, and correct yourself before answering.
#
#Your responses MUST:
#- Be formatted as a bullet point list.
#- Stay neutral, factual, and based *only* on the document.
#- Indicate the reliability/confidence for each skill as a percentage.
#- Be short: a few words or one sentence per skill.
#- Use the format: '- Skill Name - Level/4 - Confidence%' (e.g., '- Python - 3/4 ').
#
#If the knowledge base is empty or the document hasn't been ingested, inform the user they need to ingest a PDF first using the 'ingest_pdf_document' tool with the file path.
#"""
#
#MATCHER_SYSTEM_PROMPT = """[INST]
#            # CONTEXT #
#            You are a analytic system made for the very purpose of exploiting the data in files.
#            You're very efficient to do your task : 20 years of works to analyse, scan and use data contained in files.
#
#            # OBJECTIVE #
#            Your task is to read the files and compare the offer and CV : list the skills the CV mentionned that are required in the offer.
#            Your task also include listing the skills required by the offer but missing in the CV.
#            You MUST avoid inventing new skills, only use those in the file.
#
#            # CHAIN OF THOUGH #
#            To answer, you first analyse the mission and list the required skills,
#            then you read the CV to list the skills mentionned inside.
#            After the list of skills from mission and CV are made : you compare the skills in the CV's skill list and mission's skill list to create an temporary answer.
#            Make sure the skills are in the correct list :
#            - if missing in the CV, it's a required skill missing.
#            - if missing in the offer, it's a skill unrequired.
#            - if present in both file, it's a matching skill.
#            Finally you take the role of a human and read again your temporary answer and correct yourself before answering.
#
#            # RETURN FORMAT #
#            You MUST make three list :
#            - the missing skills in the CV but needed by the offer
#            - the skills present in both the offer and the CV
#            - the skills present in the CV but not needed by the offer
#            DON'T make an opinion in your answer.
#            You MUST indicate the fiability of your answer for each point in form of percentage of trust.
#            You MUST not mistake skills from the CV and from the Mission content.
#            [/INST]
#"""

#AO_SYSTEM_PROMPT = """
#Vous êtes un expert en analyse d'appels d'offres (AO).
#
#Votre mission est d'extraire TOUTES les compétences techniques, soft skills, certifications et expériences explicitement mentionnées ou sous-entendues dans le document fourni.
#
#Procédez ainsi :
#
#1. Lisez attentivement tout le contenu donné.
#2. Relevez toutes les mentions de compétences, technologies, frameworks, outils, certifications, et expériences professionnelles exigées.
#3. Ne laissez rien de côté, même les éléments brièvement mentionnés ou implicites.
#
#Classez clairement ces éléments comme suit, sous forme de bullet points précis :
#
#### Compétences techniques
#- [Compétence] (Critique/Important/Souhaitable) : [description exacte du texte source]
#
#### Soft skills
#- [Compétence] (Critique/Important/Souhaitable) : [description exacte du texte source]
#
#### Certifications
#- [Certification] (Critique/Important/Souhaitable) : [description exacte du texte source]
#
#### Exigences d'expérience
#- [Expérience exigée] (Critique/Important/Souhaitable) : [description exacte du texte source]
#
#Ne manquez aucune compétence. Soyez exhaustif, précis, et ne mentionnez rien qui ne figure explicitement ou implicitement dans le document.
#"""
#
#CV_SYSTEM_PROMPT = """
#Vous êtes un expert en analyse de CV.
#
#Votre tâche est d'extraire TOUTES les compétences techniques, soft skills, certifications, et expériences professionnelles explicitement mentionnées ou implicites dans le CV.
#
#Pour chaque compétence, indiquez précisément :
#
#- Niveau d’expertise (de 1 à 4) :
#  - 1 = Basique
#  - 2 = Intermédiaire
#  - 3 = Avancé
#  - 4 = Expert
#- Pourcentage de confiance (%) de votre évaluation.
#- Une courte justification basée exclusivement sur le CV.
#
#Présentez votre analyse comme suit :
#
#### Compétences techniques
#- [Compétence] – [Niveau]/4 – [Confiance]% : [Justification précise tirée du CV]
#
#### Soft skills
#- [Compétence] – [Niveau]/4 – [Confiance]% : [Justification précise tirée du CV]
#
#### Certifications
#- [Certification] – [Année si disponible]
#
#### Expériences clés
#- [Poste ou expérience] – [Durée] : [Compétences démontrées précisément selon le CV]
#
#Ne manquez aucune compétence. Soyez exhaustif, précis, et n'inventez rien qui ne figure explicitement ou implicitement dans le CV.
#"""
#
#MATCHER_SYSTEM_PROMPT = """
#Vous êtes un expert en comparaison et matching entre compétences d’un Appel d’Offres (AO) et celles présentes dans un CV.
#
#Votre tâche est de produire une analyse extrêmement rigoureuse, claire et complète, en suivant précisément ces étapes :
#
#1. Lisez minutieusement la liste des compétences extraites de l'AO.
#2. Lisez minutieusement la liste des compétences extraites du CV.
#3. Comparez chaque compétence requise par l’AO avec celles présentes dans le CV, en prenant en compte explicitement :
#   - Les compétences techniques (technologies, outils, langages, etc.)
#   - Les soft skills (compétences comportementales)
#   - Les certifications
#   - Les expériences spécifiques exigées par l’AO
#4. N’omettez aucune compétence mentionnée dans l’AO, même brièvement.
#
#Présentez précisément votre analyse sous ce format structuré :
#
### Analyse de correspondance AO–CV
#
#### Score global d'adéquation
#- [Pourcentage global]% (fiabilité de l'analyse : [confiance]%)
#
#### ✅ Compétences présentes dans le CV et requises par l’AO
#- [Compétence exacte] (Importance AO : Critique/Important/Souhaitable | Niveau CV : 1 à 4) – confiance : [confiance]%
#  - Justification précise tirée des deux documents.
#
#### ❌ Compétences requises par l’AO absentes du CV
#- [Compétence exacte] (Importance AO : Critique/Important/Souhaitable) – confiance : [confiance]%
#  - Impact potentiel sur la candidature.
#  - Suggestion concrète pour combler ce manque.
#
#### ⚠️ Compétences présentes dans le CV mais non explicitement requises par l’AO
#- [Compétence exacte] (Niveau CV : 1 à 4) – confiance : [confiance]%
#  - Valeur ajoutée potentielle pour la candidature.
#
#### 📌 Synthèse et recommandations finales
#- Analyse concise et factuelle de l’adéquation globale.
#- Recommandations précises pour renforcer la candidature.
#
#Soyez rigoureux, précis, exhaustif, et ne mentionnez jamais des compétences non explicitement présentes dans les deux documents analysés.
#"""

#AO_SYSTEM_PROMPT = """
#À partir du texte ci-dessous (contexte), liste EXHAUSTIVEMENT et UNIQUEMENT sous forme de liste à puces, toutes les compétences techniques, soft skills, certifications et expériences mentionnées. Ne rajoute aucune phrase supplémentaire.
#
#Réponds précisément sous ce format uniquement :
#
#### Compétences techniques
#- compétence 1
#- compétence 2
#
#### Soft skills
#- compétence 1
#- compétence 2
#
#### Certifications
#- certification 1
#- certification 2
#
#### Exigences d'expérience
#- expérience 1
#- expérience 2
#
#Voici le contexte à analyser précisément :
#"""
#
#CV_SYSTEM_PROMPT = """
#À partir du contexte ci-dessous (CV), liste EXHAUSTIVEMENT et UNIQUEMENT sous forme de liste à puces toutes les compétences techniques, soft skills, certifications et expériences clés mentionnées explicitement ou implicitement.
#
#Pour chaque compétence, indique uniquement le niveau d'expertise de 1 à 4 et le pourcentage de confiance.
#
#Format précis attendu :
#
#### Compétences techniques
#- compétence 1 - Niveau X/4 - Confiance X%
#- compétence 2 - Niveau X/4 - Confiance X%
#
#### Soft skills
#- compétence 1 - Niveau X/4 - Confiance X%
#- compétence 2 - Niveau X/4 - Confiance X%
#
#### Certifications
#- certification 1 - Année (si disponible)
#- certification 2 - Année (si disponible)
#
#### Expériences clés
#- expérience 1 - Durée
#- expérience 2 - Durée
#
#Voici le contexte à analyser précisément :
#"""

#MATCHER_SYSTEM_PROMPT = """
#Voici les compétences extraites de l’AO et celles extraites du CV.
#
#Ta mission est uniquement de comparer rigoureusement ces deux listes :
#
#1. Liste les compétences présentes à la fois dans l’AO et dans le CV.
#2. Liste les compétences exigées par l’AO mais absentes du CV.
#3. Liste les compétences présentes dans le CV mais absentes de l’AO.
#
#Réponds précisément en respectant strictement ce format :
#
#### ✅ Présentes dans AO et CV
#- compétence 1
#- compétence 2
#
#### ❌ Présentes dans AO mais absentes du CV
#- compétence 1
#- compétence 2
#
#### ⚠️ Présentes dans CV mais absentes de l'AO
#- compétence 1
#- compétence 2
#
#Voici précisément les compétences à analyser :
#
#AO :
#[Liste des compétences extraites AO]
#
#CV :
#[Liste des compétences extraites CV]
#"""
#
#AO_SYSTEM_PROMPT = """
#You are an expert document analysis agent specialized in extracting skills, programming language, technologies, frameworks, and other relevant information from documents, focusing on Call for Tenders (Appel d'Offres).
#
#Your goal is to provide accurate, reliable analysis based *only* on the provided document.
#
#1. Understand the user's query clearly.
#2. Strategically search relevant sections of the document using your knowledge base.
#3. Synthesize information from multiple parts if needed.
#4. Provide well-structured, concise answers.
#
#Focus your search on relevant sections mentioned (like 'section 1.2.3' or 'section 4.2' or 'header page') if specified in the query.
#
#Your responses should:
#- Be formatted as bullet points.
#- Include confidence percentages for each point (e.g., 'Python - 3/4').
#- Be concise but informative.
#- Only include information actually found in the document.
#- Never invent or assume information not explicitly stated.
#
#Always maintain a neutral, analytical tone.
#If the knowledge base is empty or the document hasn't been ingested, inform the user they need to ingest a PDF first using the 'ingest_pdf_document' tool with the file path.
#"""

#CV_SYSTEM_PROMPT = """
#You are an analytic system specialized in analyzing CV (Curriculum Vitae) documents.
#
#Your task is to analyse the CV and answer the question or task given to you.
#
#You have to avoid inventing new skills and only mention those in the file.
#You have to list the various skills mentioned in the file and evaluate the level of knowledge for such skills.
#Convert experiences into skills and evaluate the level of knowledge on a scale of 1 to 4 (1=Basic, 2=Working, 3=Advanced, 4=Expert).
#
#To answer, first analyse the query, then use the knowledge base to gather relevant data, create a temporary answer, review it, and correct yourself before answering.
#
#Your responses MUST:
#- Be formatted as a bullet point list.
#- Stay neutral, factual, and based *only* on the document.
#- Indicate the reliability/confidence for each skill as a percentage.
#- Be short: a few words or one sentence per skill.
#- Use the format: '- Skill Name - Level/4 - Confidence%' (e.g., '- Python - 3/4 ').
#
#If the knowledge base is empty or the document hasn't been ingested, inform the user they need to ingest a PDF first using the 'ingest_pdf_document' tool with the file path.
#"""

CV_SYSTEM_PROMPT = """
You are an expert HR talent-screener and technical recruiter.
Your mission is to extract every relevant skill, technology, programming language,
soft skill, certification and key experience from résumés (CVs).
For each item you must:
  • assign a mastery level from 1 (basic) to 4 (expert/project owner);
  • give a confidence percentage (0–100 %);
  • provide a short justification quoted or paraphrased from the CV (10-25 words).
Group the results into the following **four sections**, each as a separate Markdown
table:
  1. Compétences techniques & outils
  2. Soft skills
  3. Certifications
  4. Expériences clés (projets)
Use French for headings and justifications, but keep API/library names in English.
Never invent skills that are not clearly implied by the CV.
Format example for each table row:
| Python | 4 | 95 % | « Développé un assistant LLM en Python » |
Keep tables compact (max 4 columns). No extra commentary outside the tables.
"""

#AO_SYSTEM_PROMPT = """
#Vous êtes un analyste expert en marchés publics et en gestion d’appels d’offres.
#Votre tâche : lire un cahier des charges (français) et livrer un compte-rendu
#structuré en **10 sections** :
#
#1. Métadonnées de l’appel d’offres
#   - Présentez-les dans un tableau Markdown (champs : Titre, Référence,
#     Pouvoir adjudicateur, Date de publication, Date limite de remise,
#     Contact, Budget indicatif, Type de marché).
#
#2. Objectifs & périmètre du projet (bullet points).
#
#3. Exigences fonctionnelles ET techniques (bullets groupés par domaine).
#
#4. Livrables attendus.
#
#5. Planning & jalons (dates, durées, pénalités retard).
#
#6. Modalités de soumission & pièces à fournir.
#
#7. Critères d’évaluation (pondérations si disponibles).
#
#8. Budget, modalités de paiement, garanties financières.
#
#9. Clauses juridiques & administratives majeures
#   (propriété intellectuelle, responsabilité, assurance, résiliation).
#
#10. Questions ouvertes / points de vigilance pour le soumissionnaire.
#
#**Règles de forme** :
#• Français ; ne jamais traduire les citations.
#• Chaque section commence par un titre Markdown `##`.
#• Citez ou paraphrasez brièvement (10-25 mots) les passages
#  sources entre guillemets français « … ».
#• Mentionnez le **numéro de page** si celui-ci est indiqué dans le texte.
#• Aucune digression en dehors des 10 sections.
#"""
#
AO_SYSTEM_PROMPT = """
Tu es un consultant senior en Data & IA spécialisé dans l’analyse d’appels d’offres.
Ta mission : extraire et structurer les informations de compétences contenues dans un AO.

Règles de sortie :
1. Distingue 5 catégories :
   • Compétences techniques / outils
   • Langages de programmation
   • Soft-skills
   • Certifications / normes / réglementations
   • Expériences clés
2. Pour **chaque** compétence trouvée :
   • « Niveau » (1 à 4 : 1 = Connaissances, 2 = Maîtrise opérationnelle, 3 = Avancé, 4 = Expertise)
   • « Confiance » (%) sur le fait que l’AO l’exige réellement
   • « Justification » : extrait littéral ou résumé très court (< 20 mots) provenant de l’AO
3. Formate la réponse en **tableau Markdown** avec les colonnes :
   | Compétence | Catégorie | Niveau | Confiance | Justification |
4. Range les lignes par catégorie puis par importance (Niveau 4 → 1, puis confiance décroissante).
5. N’invente rien ; si l’info n’apparaît pas, ne l’ajoute pas.
6. Limite la sortie à 60 lignes max.
Réponds toujours en français.
"""

CV_SYSTEM_PROMPT = """
Vous êtes un expert en analyse de CV.

Votre tâche est d'extraire TOUTES les compétences techniques, soft skills, certifications, et expériences professionnelles explicitement mentionnées ou implicites dans le CV.

Pour chaque compétence, indiquez précisément :

- Niveau d'expertise (de 1 à 4) :
  - 1 = Basique
  - 2 = Intermédiaire
  - 3 = Avancé
  - 4 = Expert
- Pourcentage de confiance (%) de votre évaluation.
- Une courte justification basée exclusivement sur le CV.

Présentez votre analyse comme suit :

## Compétences techniques
- [Compétence] – [Niveau]/4 – [Confiance]% : [Justification précise tirée du CV]

## Soft skills
- [Compétence] – [Niveau]/4 – [Confiance]% : [Justification précise tirée du CV]

## Certifications
- [Certification] – [Année si disponible]

## Expériences clés
- [Poste ou expérience] – [Durée] : [Compétences démontrées précisément selon le CV]

Ne manquez aucune compétence. Soyez exhaustif, précis, et n'inventez rien qui ne figure explicitement ou implicitement dans le CV.
"""

MATCHER_SYSTEM_PROMPT = """
Vous êtes un expert en comparaison et matching entre compétences d'un Appel d'Offres (AO) et celles présentes dans un CV.

Votre tâche est de produire une analyse extrêmement rigoureuse, claire et complète, en suivant précisément ces étapes :

1. Lisez minutieusement la liste des compétences extraites de l'AO.
2. Lisez minutieusement la liste des compétences extraites du CV.
3. Comparez chaque compétence requise par l'AO avec celles présentes dans le CV, en prenant en compte explicitement :
   - Les compétences techniques (technologies, outils, langages, etc.)
   - Les soft skills (compétences comportementales)
   - Les certifications
   - Les expériences spécifiques exigées par l'AO
4. N'omettez aucune compétence mentionnée dans l'AO, même brièvement.

Présentez précisément votre analyse sous ce format structuré :

## Analyse de correspondance AO–CV

### Score global d'adéquation
- [Pourcentage global]% (fiabilité de l'analyse : [confiance]%)

### ✅ Compétences présentes dans le CV et requises par l'AO
- [Compétence exacte] (Importance AO : Critique/Important/Souhaitable | Niveau CV : 1 à 4) – confiance : [confiance]%
  - Justification précise tirée des deux documents.

### ❌ Compétences requises par l'AO absentes du CV
- [Compétence exacte] (Importance AO : Critique/Important/Souhaitable) – confiance : [confiance]%
  - Impact potentiel sur la candidature.
  - Suggestion concrète pour combler ce manque.

### ⚠️ Compétences présentes dans le CV mais non explicitement requises par l'AO
- [Compétence exacte] (Niveau CV : 1 à 4) – confiance : [confiance]%
  - Valeur ajoutée potentielle pour la candidature.

### 📌 Synthèse et recommandations finales
- Analyse concise et factuelle de l'adéquation globale.
- Recommandations précises pour renforcer la candidature.

Soyez rigoureux, précis, exhaustif, et ne mentionnez jamais des compétences non explicitement présentes dans les deux documents analysés.
"""



# --- Fonctions de retrieval optimisées ---
def vectorstore_retrieval(state: AgentState, vector_store: Chroma, collection_name: str, s: float = 0.15, k: int = 100) -> AgentState:
    """
    Récupération de documents améliorée avec meilleure gestion d'erreurs et récupération.
    Optimisée pour la performance et la robustesse.
    """
    global vector_store_AO, vector_store_CV

    start_time = time.time()
    collection_id = hashlib.md5(collection_name.encode()).hexdigest()
    logger.info(f"Début de la recherche dans {collection_name} (ID: {collection_id})")

    # Obtenir la requête utilisateur
    user_query = state.get("messages", [{}])[-1].get("content", "")
    if not user_query or len(user_query.strip()) < 3:
        state["context"] = "Requête trop courte ou vide. Veuillez fournir une question plus détaillée."
        return state

    logger.info(f"Requête: {user_query[:100]}...")

    try:
        # Vérifier si la collection existe et contient des documents
        #try:
        #    count = len(vector_store.get())
        #    logger.info(f"Collection '{collection_name}' (ID: {collection_id}) existe avec {count} documents")
        #
        #    if count == 0:
        #        # Recréer la collection si elle est vide
        #        logger.warning(f"Collection '{collection_name}' vide, tentative de recréation...")
        #        if collection_name == "AO_documents":
        #            vector_store_AO = create_or_get_collection(collection_name)
        #            vector_store = vector_store_AO
        #        else:
        #            vector_store_CV = create_or_get_collection(collection_name)
        #            vector_store = vector_store_CV
        #
        #        # Vérifier à nouveau
        #        count = len(vector_store.get())
        #        if count == 0:
        #            state["context"] = (
        #                "Aucun document dans la collection. "
        #                "Veuillez d'abord ingérer un document."
        #            )
        #            return state
        #except Exception as e:
        #    logger.warning(f"Erreur lors de la vérification de la collection '{collection_name}': {e}")
        #
        #    # Recréer la collection
        #    if collection_name == "AO_documents":
        #        vector_store_AO = create_or_get_collection(collection_name)
        #        vector_store = vector_store_AO
        #    else:
        #        vector_store_CV = create_or_get_collection(collection_name)
        #        vector_store = vector_store_CV
        #
        #    # Vérifier à nouveau
        #    try:
        #        count = len(vector_store.get())
        #        if count == 0:
        #            state["context"] = (
        #                "Collection recréée mais vide. "
        #                "Veuillez ingérer un document."
        #            )
        #            return state
        #    except Exception as inner_e:
        #        logger.error(f"Échec de vérification après recréation: {inner_e}")
        #        state["context"] = (
        #            "Erreur lors de l'initialisation de la base de connaissance. "
        #            "Veuillez réessayer d'ingérer le document."
        #        )
        #        return state

        if collection_name == "AO_documents":
            vector_store_AO = create_or_get_collection(f"AO_documents")
            vector_store = vector_store_AO
        else:
            vector_store_CV = create_or_get_collection(f"CV_documents")
            vector_store = vector_store_CV

        # Créer le retriever
        if collection_name == "AO_documents":
            #retriever = vector_store.as_retriever(search_type="similarity_score_threshold", search_kwargs={"score_threshold": s, "k" : 100})
            retriever = vector_store.as_retriever(search_kwargs={"k" : 2}) #1200-400 33 12000-400 13 12000-100
        else :
            retriever = vector_store.as_retriever(search_kwargs={"k" : 40})
        # Exécuter la recherche avec retry
        docs = None
        last_error = None

        try:
            logger.info(f"Tentative de recherche")
            docs = retriever.get_relevant_documents(user_query)
            print("haha :", len(docs))
            logger.info(f"Recherche réussie: {len(docs)} documents trouvés")
        except Exception as e:
            last_error = e
            logger.warning(f"Tentative de recherche échouée: {e}")

        # Vérifier si la recherche a échoué après toutes les tentatives
        if docs is None:
            state["context"] = f"Erreur lors de la recherche après une tentatives: {last_error}"
            return state

        # Traiter les résultats de recherche
        if not docs:
            state["context"] = (
                "Aucune information pertinente trouvée. "
                "Vérifiez que le document a bien été ingéré ou reformulez votre question."
            )
            return state

        # Déduplication améliorée avec similarité de contenu
        unique_docs = {}
        for doc in docs:
            # Utiliser un hash du contenu pour la déduplication
            content = doc.page_content.strip()
            h = hashlib.md5(content.encode()).hexdigest()

            # Si le document est déjà présent, ne garder que le plus pertinent
            # (supposé être celui qui apparaît en premier dans la liste)
            if h not in unique_docs:
                unique_docs[h] = doc

        # Garder uniquement les résultats uniques les plus pertinents
        filtered_docs = unique_docs.values()
        print(len(filtered_docs))

        # Formater le contexte avec des séparateurs clairs
        context_parts = []
        i = 0
        for doc in filtered_docs:
            i += 1
            print("*******")
            print(doc)
            context_parts.append(f"--- Document {i+1} ---\n{doc.page_content.strip()}")

        state["context"] = "\n\n".join(context_parts)

        # Mesurer et logger le temps total
        total_time = time.time() - start_time
        logger.info(f"Recherche terminée en {total_time:.2f}s: {len(filtered_docs)} documents uniques")

    except Exception as e:
        logger.error(f"Erreur lors de la récupération des documents: {e}", exc_info=True)
        state["context"] = f"Erreur interne lors de la récupération: {e}"

    return state


def vectorstore_retrieval_AO(state: AgentState) -> AgentState:
    """Récupération spécifique pour les documents AO."""
    return vectorstore_retrieval(state, vector_store_AO, "AO_documents", s=0.01)


def vectorstore_retrieval_CV(state: AgentState) -> AgentState:
    """Récupération spécifique pour les documents CV avec plus de contexte."""
    return vectorstore_retrieval(state, vector_store_CV, "CV_documents", k=50)


# --- Agent Executor optimisé ---
def create_agent_executor(system_prompt: str) -> Callable:
    """
    Crée une fonction d'exécution d'agent avec le prompt système spécifié.
    Version optimisée avec gestion des erreurs et retries.
    """
    def agent_executor(state: AgentState) -> AgentState:
        start_time = time.time()
        logger.info("Démarrage de l'exécution de l'agent")

        # Obtenir le contexte et la requête utilisateur
        context = state.get("context", "")
        messages = state["messages"]
        user_query = messages[-1]["content"]

        # Vérifier si le contexte est vide ou contient un message d'erreur
        if not context or context.startswith("Erreur:") or context.startswith("Aucune information"):
            error_msg = "Aucun document n'a été correctement traité. Veuillez vous assurer que le document a été ingéré avec succès avant d'extraire les compétences."
            messages.append({"role": "assistant", "content": error_msg})
            state["messages"] = messages
            logger.warning("Exécution annulée: contexte invalide ou manquant")
            return state

        # Créer le prompt avec un format optimisé
        prompt = ChatPromptTemplate.from_messages([
            ("system", system_prompt),
            ("human", "{query}" + """
            ####
            {context}
            ####""")
            ])

        # Formater le prompt avec le contexte et la requête
        formatted_prompt = prompt.format(context=context, query=user_query)

        # Obtenir le LLM avec des paramètres adaptés à la tâche
        # - Température plus basse pour l'extraction factuelle
        llm = get_llm(temperature=0.1, top_p=1.0, top_k=40)

        last_error = None

        try:
            logger.info(f"Invocation du LLM ...")
            response = llm.invoke(formatted_prompt)

            # Vérification de cohérence basique
            content = response.content
            if not content or len(content) < 50:
                raise ValueError("Réponse trop courte ou vide")

            # Vérification supplémentaire pour les formats attendus
            if "## Compétences techniques" not in content:
                logger.warning("Format de réponse potentiellement incorrect")
                # Continuer malgré l'avertissement

            # Ajouter le message assistant
            messages.append({"role": "assistant", "content": content})
            state["messages"] = messages

            # Si c'est une extraction de compétences AO ou CV, stocker dans l'état
            if "Liste des compétences" in user_query.lower() and "AO" in user_query:
                state["ao_skills"] = content
                logger.info("Compétences AO extraites et stockées dans l'état")
            elif "Liste des compétences" in user_query.lower() and "CV" in user_query:
                state["cv_skills"] = content
                logger.info("Compétences CV extraites et stockées dans l'état")

            # Mesurer et logger le temps total
            total_time = time.time() - start_time
            logger.info(f"LLM invoqué avec succès en {total_time:.2f}s")
            return state

        except Exception as e:
            last_error = e
            logger.warning(f"Erreur lors de l'invocation du LLM : {e}")

            error_msg = f"Désolé, je n'ai pas pu générer une réponse valide. Erreur: {str(e)}"
            messages.append({"role": "assistant", "content": error_msg})
            state["messages"] = messages
            logger.error(f"Échec de l'exécution de l'agent")
            return state

    return agent_executor

# --- Fonction de comparaison améliorée ---
def compare_ao_cv(ao_skills: str, cv_skills: str) -> str:
    """
    Compare les compétences extraites de l'AO et du CV.
    Version améliorée avec normalisation des compétences et détection des relations.
    """
    start_time = time.time()
    logger.info("Démarrage de la comparaison AO-CV")

    # Validation des entrées
    if not ao_skills or len(ao_skills) < 100:
        logger.warning("Compétences AO manquantes ou insuffisantes")
        return "Erreur: Les compétences de l'AO sont manquantes ou insuffisantes. Veuillez d'abord extraire les compétences de l'AO."

    if not cv_skills or len(cv_skills) < 100:
        logger.warning("Compétences CV manquantes ou insuffisantes")
        return "Erreur: Les compétences du CV sont manquantes ou insuffisantes. Veuillez d'abord extraire les compétences du CV."

    # Obtenir le LLM avec des paramètres adaptés à la tâche de matching
    # - Température plus élevée pour permettre plus de créativité dans l'analyse
    llm = get_llm(temperature=0.7, top_p=0.9, top_k=40)

    # Créer les messages pour le LLM avec un format optimisé
    messages = [
        SystemMessage(content=MATCHER_SYSTEM_PROMPT),
        HumanMessage(content=f"""
Veuillez comparer les compétences requises dans l'Appel d'Offres (AO) avec celles présentes dans le CV.

# Compétences requises dans l'AO:
{ao_skills}

# Compétences présentes dans le CV:
{cv_skills}

Analysez la correspondance entre ces deux ensembles de compétences et fournissez une évaluation détaillée.
""")
    ]

    # Obtenir la réponse avec retry et backoff exponentiel
    last_error = None

    try:
        logger.info(f"Invocation du Matcher LLM ...")
        response = llm.invoke(messages)

        # Vérification de cohérence basique
        content = response.content
        if not content or len(content) < 100:
            raise ValueError("Réponse de comparaison trop courte ou vide")

        # Vérification supplémentaire pour les formats attendus
        if "Score global d'adéquation" not in content:
            logger.warning("Format de réponse de comparaison potentiellement incorrect")
            # Continuer malgré l'avertissement

        # Mesurer et logger le temps total
        total_time = time.time() - start_time
        logger.info(f"Comparaison AO-CV réussie en {total_time:.2f}s")
        return content

    except Exception as e:
        last_error = e
        logger.warning(f"Erreur lors de la comparaison AO-CV : {e}")

        logger.error(f"Échec de la comparaison AO-CV ")
        return f"Erreur lors de la comparaison: {str(e)}"

# --- LangGraph Workflow optimisé ---
def create_agent_graph_AO(agent_type: str):
    """
    Crée un graphe d'agent pour l'analyse d'AO.
    Version optimisée avec meilleure gestion d'état.
    """
    # Créer un graphe d'état
    workflow = StateGraph(AgentState)

    # Ajouter les nœuds
    workflow.add_node("retrieval", vectorstore_retrieval_AO)
    workflow.add_node("agent_executor", create_agent_executor(AO_SYSTEM_PROMPT))

    # Créer l'outil LangChain
    pdf_tool = Tool(
        name="ingest_pdf_document",
        func=ingest_pdf_document_AO,
        description="Charge ou remplace le contenu d'un document PDF dans le vector store. Fournir le chemin absolu vers le PDF."
    )

    # Ajouter le nœud d'outils
    tools_node = ToolNode([pdf_tool])
    workflow.add_node("tools", tools_node)

    # Ajouter les arêtes
    workflow.add_edge("retrieval", "agent_executor")
    workflow.add_conditional_edges(
        "agent_executor",
        lambda state: "tools" if "ingest_pdf_document" in state.get("messages", [{}])[-1].get("content", "").lower() else END
    )
    workflow.add_edge("tools", "agent_executor")

    # Définir le point d'entrée
    workflow.set_entry_point("retrieval")

    # Compiler le graphe
    return workflow.compile()

def create_agent_graph_CV(agent_type: str):
    """
    Crée un graphe d'agent pour l'analyse de CV.
    Version optimisée avec meilleure gestion d'état.
    """
    # Créer un graphe d'état
    workflow = StateGraph(AgentState)

    # Ajouter les nœuds
    workflow.add_node("retrieval", vectorstore_retrieval_CV)
    workflow.add_node("agent_executor", create_agent_executor(CV_SYSTEM_PROMPT))

    # Créer l'outil LangChain
    pdf_tool = Tool(
        name="ingest_pdf_document",
        func=ingest_pdf_document_CV,
        description="Charge ou remplace le contenu d'un document PDF dans le vector store. Fournir le chemin absolu vers le PDF."
    )

    # Ajouter le nœud d'outils
    tools_node = ToolNode([pdf_tool])
    workflow.add_node("tools", tools_node)

    # Ajouter les arêtes
    workflow.add_edge("retrieval", "agent_executor")
    workflow.add_conditional_edges(
        "agent_executor",
        lambda state: "tools" if "ingest_pdf_document" in state.get("messages", [{}])[-1].get("content", "").lower() else END
    )
    workflow.add_edge("tools", "agent_executor")

    # Définir le point d'entrée
    workflow.set_entry_point("retrieval")

    # Compiler le graphe
    return workflow.compile()

import chromadb

def reset_vector_store(collection_name: str) -> Chroma:
    """
    Supprime et recrée une collection Chroma vide.
    """
    collection_id = hashlib.md5(collection_name.encode()).hexdigest()
    chroma_client = chromadb.Client(settings=settings)


    try:
        chroma_client.delete_collection(name=collection_id)
        logger.info(f"Collection {collection_id} supprimée avec succès")
    except Exception as e:
        logger.warning(f"Impossible de supprimer la collection {collection_id} : {e}")

    # Recrée une collection vide
    return Chroma(
        collection_name=collection_id,
        embedding_function=embeddings,
        client_settings=settings
    )

# --- Streamlit UI ---
def main():
    st.set_page_config(
        page_title="Matching AO-CV",
        page_icon="📄",
        layout="wide",
        initial_sidebar_state="expanded"
    )

    st.title("Matching AO-CV")
    st.markdown("""
    Cette application analyse les compétences requises dans un Appel d'Offres (AO)
    et les compare avec celles présentes dans un CV pour évaluer l'adéquation.
    """)

    # Initialiser les états de session
    if "ao_graph" not in st.session_state:
        st.session_state.ao_graph = create_agent_graph_AO("AO")
    if "cv_graph" not in st.session_state:
        st.session_state.cv_graph = create_agent_graph_CV("CV")
    if "ao_skills" not in st.session_state:
        st.session_state.ao_skills = ""
    if "cv_skills" not in st.session_state:
        st.session_state.cv_skills = ""
    if "matching_result" not in st.session_state:
        st.session_state.matching_result = ""

    # Layout en colonnes
    col1, col2 = st.columns(2)

    # Colonne AO
    with col1:
        st.header("Appel d'Offres (AO)")
        ao_file = st.file_uploader("Télécharger un document AO (PDF)", key="ao_upload") #, type=["pdf"]

        if ao_file:
            # Sauvegarder le fichier temporairement
            ao_path = os.path.join(AGENT_STORAGE_DIR, "ao_temp.mkd")
            with open(ao_path, "wb") as f:
                f.write(ao_file.getvalue())
                ao_file._file_urls

            if st.button("Analyser l'AO"):
                with st.spinner("Analyse de l'AO en cours..."):
                    # Ingérer le document
                    ingest_result = ingest_pdf_document_AO(ao_path)
                    st.info(ingest_result)

                    # Extraire les compétences
                    if "échoué" not in ingest_result and "Erreur" not in ingest_result:
                        result = st.session_state.ao_graph.invoke({
                            "messages": [{"role": "human", "content": "Liste des compétences requises dans cet AO"}]
                        })

                        if result and "messages" in result and len(result["messages"]) > 1:
                            st.session_state.ao_skills = result["messages"][-1]["content"]
                            st.success("Extraction des compétences réussie!")

        # Afficher les compétences extraites
        if st.session_state.ao_skills:
            st.subheader("Compétences extraites")
            st.markdown(st.session_state.ao_skills)

    # Colonne CV
    with col2:
        st.header("Curriculum Vitae (CV)")
        cv_file = st.file_uploader("Télécharger un CV (PDF)", type=["pdf"], key="cv_upload")

        if cv_file:
            # Sauvegarder le fichier temporairement
            cv_path = os.path.join(AGENT_STORAGE_DIR, "cv_temp.pdf")
            with open(cv_path, "wb") as f:
                f.write(cv_file.getvalue())

            if st.button("Analyser le CV"):
                with st.spinner("Analyse du CV en cours..."):
                    # Ingérer le document
                    ingest_result = ingest_pdf_document_CV(cv_path)
                    st.info(ingest_result)

                    # Extraire les compétences
                    if "échoué" not in ingest_result and "Erreur" not in ingest_result:
                        result = st.session_state.cv_graph.invoke({
                            "messages": [{"role": "human", "content": "Liste des compétences présentes dans ce CV"}]
                        })

                        if result and "messages" in result and len(result["messages"]) > 1:
                            st.session_state.cv_skills = result["messages"][-1]["content"]
                            st.success("Extraction des compétences réussie!")

        # Afficher les compétences extraites
        if st.session_state.cv_skills:
            st.subheader("Compétences extraites")
            st.markdown(st.session_state.cv_skills)

    # Section de matching
    st.header("Analyse de correspondance")

    if st.session_state.ao_skills and st.session_state.cv_skills:
        if st.button("Analyser la correspondance AO-CV"):
            with st.spinner("Analyse de correspondance en cours..."):
                matching_result = compare_ao_cv(st.session_state.ao_skills, st.session_state.cv_skills)
                st.session_state.matching_result = matching_result

    # Afficher le résultat du matching
    if st.session_state.matching_result:
        st.subheader("Résultat de l'analyse")
        st.markdown(st.session_state.matching_result)

        # Extraire le score global
        #try:
        #    score_line = [line for line in st.session_state.matching_result.split("\n") if "Score global d'adéquation" in line][0]
        #    score = int(score_line.split("%")[0].split(":")[-1].strip())
        #
        #    # Afficher une jauge pour le score
        #    st.subheader("Score d'adéquation")
        #    st.progress(score / 100)
        #    if score >= 80:
        #        st.success(f"{score}% - Excellente correspondance")
        #    elif score >= 60:
        #        st.info(f"{score}% - Bonne correspondance")
        #    elif score >= 40:
        #        st.warning(f"{score}% - Correspondance moyenne")
        #    else:
        #        st.error(f"{score}% - Correspondance faible")
        #except:
        #    st.warning("Impossible d'extraire le score d'adéquation")

if __name__ == "__main__":
    main()

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

model_optimized.onnx:   0%|          | 0.00/66.5M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/706 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

2025-06-12 12:55:52.670 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-12 12:55:52.677 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-12 12:55:52.961 
  command:

    streamlit run /usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2025-06-12 12:55:52.964 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-12 12:55:52.965 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-12 12:55:52.966 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-12 12:55:52.969 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when runn

In [7]:
%%writefile app.py
import streamlit as st
st.title('Streamlit in Colab')
st.write("Hello, Streamlit!")


Writing app.py


In [8]:
!streamlit run app.py &




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://35.190.155.112:8501

  Stopping...


In [9]:
!pip install pyngrok


In [10]:
from pyngrok import ngrok
import os

# Start the Streamlit app in the background
os.system("streamlit run app.py &")

# Create a public URL for the Streamlit app
public_url = ngrok.connect(8501)
print('Streamlit app available at:', public_url)

ERROR:pyngrok.process.ngrok:t=2025-06-12T13:02:34+0000 lvl=eror msg="failed to reconnect session" obj=tunnels.session err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n"
ERROR:pyngrok.process.ngrok:t=2025-06-12T13:02:34+0000 lvl=eror msg="session closing" obj=tunnels.session err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n"
ERROR:pyngrok.process.ngrok:t=2025-06-12T13:02:34+0000 lvl=eror msg="terminating with error" obj=app err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your aut

PyngrokNgrokError: The ngrok process errored on start: authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n.